# MF-DMA (Multifractal Detrended Moving Average) Module

Reference:

Gu, Gao-Feng Gu, and Wei-Xing Zhou. "Detrending moving average algorithm for multifractals." Physical Review E, no. 82, 2010, pp. 011136,

Arianos, Sergio, and Anna Carbone. "Detrending moving average algorithm: A closed-form approximation of the scaling law." Physica A, no. 382, 2007, pp. 9–15,

Wang, Yudong, et al. “Multifractal Detrending Moving Average Analysis on The US Dollar Exchange Rates.” Physica A, no. 390, 2011, pp. 3512–3523.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import statsmodels.api as sm

In [2]:
files = 'https://raw.githubusercontent.com/y3kiys4/wqu_capstone_june2023_grp3626/main/SPX_1m_May2022_Nov2022.csv'

df = pd.read_csv(files)
# Set timestamp as index
df = df.set_index(df['Time'])
df.drop(['Time'], axis=1, inplace=True)
df.index = pd.to_datetime(df.index)

# Calculate log return
df['log_ret'] = 10000 * np.log(df.Price).diff() # 10,000 times log return

# Calculate the time difference in mininutes
df['deltaT'] = (df.index.to_series().diff().dt.days.mul(60*24, fill_value=0) +
                df.index.to_series().diff().dt.seconds.div(60, fill_value=0)
               )
df['missingT'] = df['deltaT'] - 1

df_1min = df[df['deltaT'] == 1]
df_1min['log_ret_sq'] = np.square(df_1min['log_ret'])
df_1min

C:\Users\cutef\AppData\Local\Temp\ipykernel_18532\562895779.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1min['log_ret_sq'] = np.square(df_1min['log_ret'])


,Price,log_ret,deltaT,missingT,log_ret_sq
Time,,,,,
2022-05-01 18:01:00,4146.245,14.020091,1.0,0.0,196.562943
2022-05-01 18:02:00,4147.945,4.099255,1.0,0.0,16.803892
2022-05-01 18:03:00,4151.999,9.768741,1.0,0.0,95.428308
2022-05-01 18:04:00,4151.351,-1.560816,1.0,0.0,2.436146
2022-05-01 18:05:00,4150.936,-0.999725,1.0,0.0,0.999449
...,...,...,...,...,...
2022-11-30 23:55:00,4082.990,0.000000,1.0,0.0,0.000000
2022-11-30 23:56:00,4082.749,-0.590271,1.0,0.0,0.348420
2022-11-30 23:57:00,4082.761,0.029392,1.0,0.0,0.000864


In [3]:
# Change the setting to show 1000 rows
pd.set_option('display.max_rows', 1000)

## Slice dataset into daily, weekly and monthly segments

### Determine how many data points in a week on average

In [4]:
available_weekly =  df_1min['deltaT'].resample('1w').count().to_frame()

available_weekly

,deltaT
Time,
2022-05-01,359
2022-05-08,6628
2022-05-15,6657
2022-05-22,6662
2022-05-29,6660
2022-06-05,6443
2022-06-12,6614
2022-06-19,6670
2022-06-26,6468


#### Method one: drop incomplete week data points

In [5]:
# Drop incomplete weeks

available_weekly_drop_1 = available_weekly.loc[available_weekly['deltaT']!=available_weekly['deltaT'].min()]
available_weekly_drop_2 = available_weekly_drop_1.loc[available_weekly_drop_1['deltaT']!=available_weekly_drop_1['deltaT'].min()]

available_weekly_drop_2

,deltaT
Time,
2022-05-08,6628
2022-05-15,6657
2022-05-22,6662
2022-05-29,6660
2022-06-05,6443
2022-06-12,6614
2022-06-19,6670
2022-06-26,6468
2022-07-03,6645


In [6]:
avg_weekly1 = int(available_weekly_drop_2.mean())
avg_weekly1

C:\Users\cutef\AppData\Local\Temp\ipykernel_18532\3540125684.py:1: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  avg_weekly1 = int(available_weekly_drop_2.mean())


6593

#### Method Two: don't drop incomplete weekly data points

In [7]:
avg_weekly = int(available_weekly.mean())
avg_weekly

C:\Users\cutef\AppData\Local\Temp\ipykernel_18532\2549662872.py:1: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  avg_weekly = int(available_weekly.mean())


6317

### Determine how many data points in a day on average

In [8]:
available_daily = df_1min['deltaT'].resample('1d').count().to_frame()
available_daily['day_of_week'] = available_daily.index.dayofweek
print(available_daily)

            deltaT  day_of_week
Time                           
2022-05-01     359            6
2022-05-02    1330            0
2022-05-03    1321            1
2022-05-04    1325            2
2022-05-05    1318            3
2022-05-06     975            4
2022-05-07       0            5
2022-05-08     359            6
2022-05-09    1332            0
2022-05-10    1325            1
2022-05-11    1332            2
2022-05-12    1334            3
2022-05-13     975            4
2022-05-14       0            5
2022-05-15     359            6
2022-05-16    1330            0
2022-05-17    1330            1
2022-05-18    1334            2
2022-05-19    1334            3
2022-05-20     975            4
2022-05-21       0            5
2022-05-22     359            6
2022-05-23    1334            0
2022-05-24    1328            1
2022-05-25    1332            2
2022-05-26    1334            3
2022-05-27     975            4
2022-05-28       0            5
2022-05-29     357            6
2022-05-

We can combine Fridays and Sundays data to align with the other weekdays

In [9]:
# Average daily data availability
avg_daily = int(available_daily['deltaT'].mean() * 7 / 5) # No available data on Sundays
avg_daily

1322

### Determine how many data points in a month on average

In [10]:
available_monthly =  df_1min['deltaT'].resample('1m').count().to_frame()
available_monthly

,deltaT
Time,
2022-05-31,29434
2022-06-30,29038
2022-07-31,27721
2022-08-31,30162
2022-09-30,28671
2022-10-31,28416
2022-11-30,28716


In [11]:
avg_monthly = available_monthly['deltaT'][:-2].mean() # Discard December 2022 because of data incompleteness
avg_monthly

29005.2

### Determine conversion ratio between daily, weekly and monthly

In [12]:
ratio_weekly = int(round(avg_weekly / avg_daily, 0))
ratio_monthly = int(round(avg_monthly / avg_daily, 0))

print(f'Conversion ratio of weekly and monthly to daily are {ratio_weekly} and {ratio_monthly}, respectively')

Conversion ratio of weekly and monthly to daily are 5 and 22, respectively


In [13]:
modified_weekly_data = int(ratio_weekly * avg_daily)
modified_monthly_data = int(ratio_monthly * avg_daily)

print(f'We use {avg_daily}, {modified_weekly_data} \
and {modified_monthly_data} data points, respectively for daily, weekly and monthly available dataset')

We use 1322, 6610 and 29084 data points, respectively for daily, weekly and monthly available dataset


In [14]:
# Change the setting to show 200 rows
pd.set_option('display.max_rows', 200)

#### Prepare the dataframe of the modified dataset

In [15]:
modified_data = df_1min['log_ret_sq'].copy()

number_modified_data = len(modified_data) // avg_daily * avg_daily
modified_data = modified_data[-number_modified_data:]
modified_data

Time
2022-05-02 14:15:00    71.111366
2022-05-02 14:16:00     8.007312
2022-05-02 14:17:00     8.640116
2022-05-02 14:18:00     0.255835
2022-05-02 14:19:00    27.702594
                         ...    
2022-11-30 23:55:00     0.000000
2022-11-30 23:56:00     0.348420
2022-11-30 23:57:00     0.000864
2022-11-30 23:58:00     0.396215
2022-11-30 23:59:00     0.082110
Name: log_ret_sq, Length: 200944, dtype: float64

#### Determine how many observations in our final analysis

In [16]:
num_obs = int(len(modified_data) // avg_daily - ratio_monthly + 1)
num_obs

131

#### Resize the modified data into observation-numbered series

In [17]:
monthly_resized_data = pd.DataFrame(index=np.arange(modified_monthly_data), columns=np.arange(num_obs))

starttime_total = time.time()
       
for j in range(num_obs): 
    loc_start_modified_data = -((-j + num_obs + ratio_monthly - 1) * avg_daily)

    for i in range(modified_monthly_data):
        monthly_resized_data[j][i] = modified_data[loc_start_modified_data]
        loc_start_modified_data += 1
    
    if (j+1) % 10 == 0:
        endtime_total = time.time()
        total_time = endtime_total - starttime_total
        print(f'{j+1} out of {num_obs} monthly columns prepared,', 
              f'total time spent till now is {int((endtime_total-starttime_total)//60)} min(s)', 
              f'{int((endtime_total-starttime_total) % 60)} sec(s).')

endtime_total = time.time()        
total_time = endtime_total - starttime_total
print(f'All monthly columns prepared!',
      f'Total time spent is {int((endtime_total-starttime_total)//60)} min',
      f'{int((endtime_total-starttime_total) % 60)} sec(s).')

10 out of 131 monthly columns prepared, total time spent till now is 0 min(s) 35 sec(s).
20 out of 131 monthly columns prepared, total time spent till now is 1 min(s) 10 sec(s).
30 out of 131 monthly columns prepared, total time spent till now is 1 min(s) 44 sec(s).
40 out of 131 monthly columns prepared, total time spent till now is 2 min(s) 19 sec(s).
50 out of 131 monthly columns prepared, total time spent till now is 2 min(s) 55 sec(s).
60 out of 131 monthly columns prepared, total time spent till now is 3 min(s) 31 sec(s).
70 out of 131 monthly columns prepared, total time spent till now is 4 min(s) 7 sec(s).
80 out of 131 monthly columns prepared, total time spent till now is 4 min(s) 43 sec(s).
90 out of 131 monthly columns prepared, total time spent till now is 5 min(s) 19 sec(s).
100 out of 131 monthly columns prepared, total time spent till now is 5 min(s) 54 sec(s).
110 out of 131 monthly columns prepared, total time spent till now is 6 min(s) 30 sec(s).
120 out of 131 month

In [18]:
monthly_resized_data

,0,1,2,3,4,5,6,7,8,9,...,121,122,123,124,125,126,127,128,129,130
0,71.111366,0.209118,128.50794,16.787078,72.324201,187.233906,38.30059,246.579329,45.34947,1.644969,...,0.129363,23.028574,13.78011,3.808376,10.773752,0.005754,0.019696,3.986904,0.0,0.38653
1,8.007312,5.630619,32.931311,9.146669,62.709439,5.580177,8.903187,0.056713,38.221926,62.739891,...,0.498446,16.294054,1.717811,18.124947,18.243348,1.360851,1.621956,0.293114,0.000059,0.446426
2,8.640116,70.671399,8.384588,529.45694,117.497259,23.89302,0.969221,23.378483,304.607129,26.337527,...,18.480609,11.212626,3.984852,1.710135,8.34365,7.543762,1.689269,0.018143,3.434759,0.001497
3,0.255835,5.711316,3.708418,189.947924,46.122058,35.622106,59.045768,160.674148,26.695085,9.415659,...,0.446785,0.0,0.338013,0.000063,3.546191,2.665439,0.0,27.971905,1.608056,1.565532
4,27.702594,27.830699,463.910376,45.345558,37.082814,45.541844,38.027716,36.450201,24.84114,3.809879,...,3.925007,11.701958,4.970166,8.700266,9.657414,10.814294,0.026823,0.001571,3.696914,0.000959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29079,5.413135,23.348845,5.875496,1.549912,23.697807,2.83922,1.812147,6.547971,1.852901,1.857592,...,3.608703,1.544732,0.025442,9.552949,152.278487,3.343653,0.016282,0.404322,3.498663,0.0
29080,31.49,1.464195,2.72496,1.409163,1.511596,3.917008,0.017403,18.797638,71.206041,5.095757,...,28.56779,18.774283,0.261546,5.97008,18.895094,0.392914,3.520462,0.000101,0.388643,0.34842
29081,87.880289,3.647274,28.362581,5.847293,5.939227,10.24276,73.298547,72.68405,3.547382,29.743824,...,12.389503,41.379097,1.596172,5.478482,0.469756,0.374545,0.57833,0.864776,0.001433,0.000864
29082,116.606859,2.907119,43.076684,0.515946,117.400615,0.221835,2.879914,17.577383,34.531739,12.328013,...,0.434701,0.642632,13.980805,0.010559,0.373137,0.402336,0.372576,0.00091,4.025076,0.396215


In [19]:
weekly_resized_data = pd.DataFrame(index=np.arange(modified_weekly_data), columns=np.arange(num_obs))

starttime_total = time.time()
       
for j in range(num_obs): 
    loc_start_modified_data = -((-j + num_obs + ratio_weekly - 1) * avg_daily)
    
    for i in range(modified_weekly_data):
        weekly_resized_data[j][i] = modified_data[loc_start_modified_data]
        loc_start_modified_data += 1
    
    if (j+1) % 10 == 0:
        endtime_total = time.time()
        total_time = endtime_total - starttime_total
        print(f'{j+1} out of {num_obs} weekly columns prepared,',
              f'total time spent till now is {int((endtime_total-starttime_total)//60)} min(s)', 
              f'{int((endtime_total-starttime_total) % 60)} sec(s).')

endtime_total = time.time()        
total_time = endtime_total - starttime_total
print(f'All weekly columns prepared!',
      f'Total time spent is {int((endtime_total-starttime_total)//60)} min',
      f'{int((endtime_total-starttime_total) % 60)} sec(s).')

10 out of 131 weekly columns prepared, total time spent till now is 0 min(s) 5 sec(s).
20 out of 131 weekly columns prepared, total time spent till now is 0 min(s) 10 sec(s).
30 out of 131 weekly columns prepared, total time spent till now is 0 min(s) 16 sec(s).
40 out of 131 weekly columns prepared, total time spent till now is 0 min(s) 21 sec(s).
50 out of 131 weekly columns prepared, total time spent till now is 0 min(s) 27 sec(s).
60 out of 131 weekly columns prepared, total time spent till now is 0 min(s) 32 sec(s).
70 out of 131 weekly columns prepared, total time spent till now is 0 min(s) 38 sec(s).
80 out of 131 weekly columns prepared, total time spent till now is 0 min(s) 44 sec(s).
90 out of 131 weekly columns prepared, total time spent till now is 0 min(s) 49 sec(s).
100 out of 131 weekly columns prepared, total time spent till now is 0 min(s) 55 sec(s).
110 out of 131 weekly columns prepared, total time spent till now is 1 min(s) 1 sec(s).
120 out of 131 weekly columns pr

In [20]:
weekly_resized_data

,0,1,2,3,4,5,6,7,8,9,...,121,122,123,124,125,126,127,128,129,130
0,5.752994,5.970206,8.557028,2.779768,5.639659,13.171149,5.580517,0.23631,1.411071,0.284226,...,14.628843,1.491534,3.46287,3.673909,0.414842,12.494416,6.5393,5.01426,0.411395,0.0
1,33.413222,0.001982,0.000212,22.53126,3.531361,85.100356,0.267426,13.341419,8.55685,8.377972,...,86.966666,21.215247,25.067593,64.783508,1.439162,0.893057,0.40556,0.905215,1.588139,22.842914
2,25.503274,1.421449,3.679338,19.828536,18.762537,8.307518,12.85593,30.841752,9.38259,5.81495,...,0.000314,0.788562,0.413831,31.59356,4.214658,0.000058,33.323274,39.306031,1.51289,1.740986
3,77.731886,33.715762,13.329835,0.116769,107.009487,5.894893,2.881878,0.480029,9.38259,3.775956,...,24.698654,3.577201,17.703739,0.432987,0.398474,0.38314,8.788849,19.066371,6.386508,30.527634
4,1.57667,0.026624,23.559571,0.214914,1.539153,34.980546,4.052083,1.512848,52.635276,2.896063,...,57.920365,0.000056,0.027532,6.063023,15.284759,14.409722,47.022371,1.2065,0.364233,3.323953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6605,5.413135,23.348845,5.875496,1.549912,23.697807,2.83922,1.812147,6.547971,1.852901,1.857592,...,3.608703,1.544732,0.025442,9.552949,152.278487,3.343653,0.016282,0.404322,3.498663,0.0
6606,31.49,1.464195,2.72496,1.409163,1.511596,3.917008,0.017403,18.797638,71.206041,5.095757,...,28.56779,18.774283,0.261546,5.97008,18.895094,0.392914,3.520462,0.000101,0.388643,0.34842
6607,87.880289,3.647274,28.362581,5.847293,5.939227,10.24276,73.298547,72.68405,3.547382,29.743824,...,12.389503,41.379097,1.596172,5.478482,0.469756,0.374545,0.57833,0.864776,0.001433,0.000864
6608,116.606859,2.907119,43.076684,0.515946,117.400615,0.221835,2.879914,17.577383,34.531739,12.328013,...,0.434701,0.642632,13.980805,0.010559,0.373137,0.402336,0.372576,0.00091,4.025076,0.396215


In [21]:
daily_resized_data = pd.DataFrame(index=np.arange(avg_daily), columns=np.arange(num_obs))

starttime_total = time.time()
       
for j in range(num_obs): 
    loc_start_modified_data = -((-j + num_obs) * avg_daily)

    for i in range(avg_daily):
        daily_resized_data[j][i] = modified_data[loc_start_modified_data]
        loc_start_modified_data += 1
    
    if (j+1) % 10 == 0:
        endtime_total = time.time()
        total_time = endtime_total - starttime_total
        print(f'{j+1} out of {num_obs} daily columns prepared,',
              f'total time spent till now is {int((endtime_total-starttime_total)//60)} min(s)', 
              f'{int((endtime_total-starttime_total) % 60)} sec(s).')

endtime_total = time.time()        
total_time = endtime_total - starttime_total
print(f'All daily columns prepared!',
      f'Total time spent is {int((endtime_total-starttime_total)//60)} min',
      f'{int((endtime_total-starttime_total) % 60)} sec(s).')

10 out of 131 daily columns prepared, total time spent till now is 0 min(s) 0 sec(s).
20 out of 131 daily columns prepared, total time spent till now is 0 min(s) 1 sec(s).
30 out of 131 daily columns prepared, total time spent till now is 0 min(s) 2 sec(s).
40 out of 131 daily columns prepared, total time spent till now is 0 min(s) 3 sec(s).
50 out of 131 daily columns prepared, total time spent till now is 0 min(s) 4 sec(s).
60 out of 131 daily columns prepared, total time spent till now is 0 min(s) 5 sec(s).
70 out of 131 daily columns prepared, total time spent till now is 0 min(s) 6 sec(s).
80 out of 131 daily columns prepared, total time spent till now is 0 min(s) 7 sec(s).
90 out of 131 daily columns prepared, total time spent till now is 0 min(s) 8 sec(s).
100 out of 131 daily columns prepared, total time spent till now is 0 min(s) 9 sec(s).
110 out of 131 daily columns prepared, total time spent till now is 0 min(s) 10 sec(s).
120 out of 131 daily columns prepared, total time s

In [22]:
daily_resized_data

,0,1,2,3,4,5,6,7,8,9,...,121,122,123,124,125,126,127,128,129,130
0,5.639659,13.171149,5.580517,0.23631,1.411071,0.284226,17.140199,124.193346,65.946614,0.393772,...,0.414842,12.494416,6.5393,5.01426,0.411395,0.0,0.000221,0.363471,1.523113,0.027736
1,3.531361,85.100356,0.267426,13.341419,8.55685,8.377972,17.32156,3.505865,6.40746,52.481979,...,1.439162,0.893057,0.40556,0.905215,1.588139,22.842914,0.000055,1.583627,0.002786,0.467602
2,18.762537,8.307518,12.85593,30.841752,9.38259,5.81495,1.526235,0.001345,14.600628,62.302297,...,4.214658,0.000058,33.323274,39.306031,1.51289,1.740986,1.46155,1.564795,0.38537,0.447125
3,107.009487,5.894893,2.881878,0.480029,9.38259,3.775956,6.069757,10.337295,47.567473,63.156671,...,0.398474,0.38314,8.788849,19.066371,6.386508,30.527634,0.0,0.363516,0.000512,0.001433
4,1.539153,34.980546,4.052083,1.512848,52.635276,2.896063,6.694964,12.301149,3.157493,33.878892,...,15.284759,14.409722,47.022371,1.2065,0.364233,3.323953,0.383648,0.372668,0.000227,0.000229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317,5.413135,23.348845,5.875496,1.549912,23.697807,2.83922,1.812147,6.547971,1.852901,1.857592,...,3.608703,1.544732,0.025442,9.552949,152.278487,3.343653,0.016282,0.404322,3.498663,0.0
1318,31.49,1.464195,2.72496,1.409163,1.511596,3.917008,0.017403,18.797638,71.206041,5.095757,...,28.56779,18.774283,0.261546,5.97008,18.895094,0.392914,3.520462,0.000101,0.388643,0.34842
1319,87.880289,3.647274,28.362581,5.847293,5.939227,10.24276,73.298547,72.68405,3.547382,29.743824,...,12.389503,41.379097,1.596172,5.478482,0.469756,0.374545,0.57833,0.864776,0.001433,0.000864
1320,116.606859,2.907119,43.076684,0.515946,117.400615,0.221835,2.879914,17.577383,34.531739,12.328013,...,0.434701,0.642632,13.980805,0.010559,0.373137,0.402336,0.372576,0.00091,4.025076,0.396215


In [23]:
# Prepare the final dataframe to analyze
final_data_cols = ['H_Daily', 'H_Weekly', 'H_Monthly', 'Real_VIX', 'Estimate__VIX']
final_data = pd.DataFrame(index=np.arange(num_obs), columns=final_data_cols)
final_data

,H_Daily,H_Weekly,H_Monthly,Real_VIX,Estimate__VIX
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


# Analyze daily cleaned data

In [24]:
daily_resized_data

,0,1,2,3,4,5,6,7,8,9,...,121,122,123,124,125,126,127,128,129,130
0,5.639659,13.171149,5.580517,0.23631,1.411071,0.284226,17.140199,124.193346,65.946614,0.393772,...,0.414842,12.494416,6.5393,5.01426,0.411395,0.0,0.000221,0.363471,1.523113,0.027736
1,3.531361,85.100356,0.267426,13.341419,8.55685,8.377972,17.32156,3.505865,6.40746,52.481979,...,1.439162,0.893057,0.40556,0.905215,1.588139,22.842914,0.000055,1.583627,0.002786,0.467602
2,18.762537,8.307518,12.85593,30.841752,9.38259,5.81495,1.526235,0.001345,14.600628,62.302297,...,4.214658,0.000058,33.323274,39.306031,1.51289,1.740986,1.46155,1.564795,0.38537,0.447125
3,107.009487,5.894893,2.881878,0.480029,9.38259,3.775956,6.069757,10.337295,47.567473,63.156671,...,0.398474,0.38314,8.788849,19.066371,6.386508,30.527634,0.0,0.363516,0.000512,0.001433
4,1.539153,34.980546,4.052083,1.512848,52.635276,2.896063,6.694964,12.301149,3.157493,33.878892,...,15.284759,14.409722,47.022371,1.2065,0.364233,3.323953,0.383648,0.372668,0.000227,0.000229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317,5.413135,23.348845,5.875496,1.549912,23.697807,2.83922,1.812147,6.547971,1.852901,1.857592,...,3.608703,1.544732,0.025442,9.552949,152.278487,3.343653,0.016282,0.404322,3.498663,0.0
1318,31.49,1.464195,2.72496,1.409163,1.511596,3.917008,0.017403,18.797638,71.206041,5.095757,...,28.56779,18.774283,0.261546,5.97008,18.895094,0.392914,3.520462,0.000101,0.388643,0.34842
1319,87.880289,3.647274,28.362581,5.847293,5.939227,10.24276,73.298547,72.68405,3.547382,29.743824,...,12.389503,41.379097,1.596172,5.478482,0.469756,0.374545,0.57833,0.864776,0.001433,0.000864
1320,116.606859,2.907119,43.076684,0.515946,117.400615,0.221835,2.879914,17.577383,34.531739,12.328013,...,0.434701,0.642632,13.980805,0.010559,0.373137,0.402336,0.372576,0.00091,4.025076,0.396215


In [25]:
# Define parameters
n_sample = avg_daily
scales = np.unique(np.logspace(np.log10(3), np.log10((n_sample+1)/10), 50).astype(int))
# qs = list(np.linspace(-10, 10, num=41, endpoint=True))
qs = [2.0]

daily_cumsum = daily_resized_data.cumsum(axis=0)
daily_cumsum

,0,1,2,3,4,5,6,7,8,9,...,121,122,123,124,125,126,127,128,129,130
0,5.639659,13.171149,5.580517,0.23631,1.411071,0.284226,17.140199,124.193346,65.946614,0.393772,...,0.414842,12.494416,6.5393,5.01426,0.411395,0.0,0.000221,0.363471,1.523113,0.027736
1,9.17102,98.271505,5.847943,13.577729,9.967921,8.662198,34.461759,127.699211,72.354074,52.875751,...,1.854004,13.387474,6.94486,5.919475,1.999534,22.842914,0.000276,1.947099,1.525898,0.495338
2,27.933556,106.579022,18.703873,44.419481,19.350511,14.477147,35.987994,127.700556,86.954702,115.178049,...,6.068662,13.387532,40.268134,45.225507,3.512424,24.5839,1.461826,3.511894,1.911268,0.942463
3,134.943043,112.473915,21.585751,44.89951,28.733101,18.253103,42.057751,138.037851,134.522175,178.334719,...,6.467136,13.770672,49.056983,64.291878,9.898931,55.111534,1.461826,3.875409,1.91178,0.943895
4,136.482196,147.454462,25.637834,46.412358,81.368377,21.149166,48.752715,150.339,137.679668,212.213611,...,21.751895,28.180395,96.079354,65.498378,10.263164,58.435488,1.845474,4.248077,1.912007,0.944124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317,21422.639157,19216.678707,17990.562672,14200.613697,15544.972845,11135.505418,17494.419955,38633.714654,37371.389585,38870.193384,...,12311.608238,11263.073322,8434.535516,6067.987764,7967.433729,1756.997371,2652.631942,6663.675621,7841.092644,15954.49148
1318,21454.129157,19218.142902,17993.287632,14202.02286,15546.484441,11139.422426,17494.437358,38652.512292,37442.595626,38875.289141,...,12340.176028,11281.847605,8434.797062,6073.957844,7986.328823,1757.390285,2656.152404,6663.675722,7841.481287,15954.8399
1319,21542.009446,19221.790175,18021.650213,14207.870154,15552.423668,11149.665186,17567.735906,38725.196342,37446.143007,38905.032965,...,12352.565531,11323.226701,8436.393235,6079.436326,7986.79858,1757.76483,2656.730734,6664.540498,7841.482721,15954.840764
1320,21658.616306,19224.697294,18064.726897,14208.386099,15669.824283,11149.887021,17570.61582,38742.773725,37480.674746,38917.360978,...,12353.000232,11323.869333,8450.37404,6079.446886,7987.171717,1758.167166,2657.10331,6664.541407,7845.507797,15955.236979


## In the for-loop:
### 1. Construct the sequence of cumulative sums
### 2. Calculate moving average
### 3. Detrend
### 4. Define non-overlapping segments and calculate each fluctuation variance
### 5. Calculate q-th order fluctuation function

In [26]:
# Create DMA flucuation function dataframe
F_summary_columns = []
for q in qs:
    for col in range(num_obs):
        F_summary_columns.append('q_'+str(q)+'_'+str(col))

F_summary_index = []
for scale in scales:
    F_summary_index.append('s_'+str(scale))

F_summary_daily = pd.DataFrame(index=F_summary_index, columns=F_summary_columns)
F_summary_daily

,q_2.0_0,q_2.0_1,q_2.0_2,q_2.0_3,q_2.0_4,q_2.0_5,q_2.0_6,q_2.0_7,q_2.0_8,q_2.0_9,...,q_2.0_121,q_2.0_122,q_2.0_123,q_2.0_124,q_2.0_125,q_2.0_126,q_2.0_127,q_2.0_128,q_2.0_129,q_2.0_130
s_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
starttime_total = time.time()

log10_scale = np.log10(scales)

for col in range(num_obs):
    ana_temp = daily_cumsum[col]
    
    for scale in scales:
        # Calculate moving average
        mean_temp = []
        for i in range(n_sample-scale+1):
            mean_temp.append(np.mean(ana_temp[i:i+scale]))
        mean_temp.reverse()
        
        # Detrend
        diff_temp_subtractor = ana_temp[:scale-2:-1].values
        diff_temp = np.subtract(np.array(mean_temp), diff_temp_subtractor)
        
        # Slice into non-overlapping segments
        segs_temp = pd.DataFrame(np.resize(diff_temp, (len(diff_temp) // scale, scale))).T
        
        # Calculate fluctuation variance
        F_seg_sq_temp = np.array(np.var(segs_temp).mean(axis=0))
        
        # Calculate the qth order fluctuation function
        for q in qs:
            if q != 0.0:
                F_temp =  ((1 / scale) * (F_seg_sq_temp ** (q / 2)).sum()) ** (1 / q)
            else:
                F_temp = np.exp(np.log(F_seg_sq_temp ** (1 / 2)).mean()) # L’Hôspital’s rule when q = 0
            
            F_summary_daily.loc[str('s_'+str(scale))][str('q_'+str(q))+'_'+str(col)] = F_temp
       
    # Calculate Hurst exponent
    log10_F = np.log10(np.array(F_summary_daily[str('q_'+str(q))+'_'+str(col)]).astype(float))
    hurst_hat = np.polyfit(log10_scale, log10_F, 1)[0]
    final_data['H_Daily'][col] = hurst_hat
    
    if (col+1) % 10 == 0:
        endtime_total = time.time()
        total_time = endtime_total - starttime_total
        print(f'{col+1} out of {num_obs} columns are done,', f'total time spent till now is',
              f'{int((endtime_total-starttime_total)//60)} min(s) {int((endtime_total-starttime_total) % 60)} sec(s).')

endtime_total = time.time()
total_time = endtime_total - starttime_total
print(f'All done! Total time spent is',f'{int((endtime_total-starttime_total)//60)}',
      f'min(s) {int((endtime_total-starttime_total) % 60)} sec(s).')

10 out of 131 columns are done, total time spent till now is 0 min(s) 59 sec(s).
20 out of 131 columns are done, total time spent till now is 1 min(s) 58 sec(s).
30 out of 131 columns are done, total time spent till now is 2 min(s) 56 sec(s).
40 out of 131 columns are done, total time spent till now is 3 min(s) 55 sec(s).
50 out of 131 columns are done, total time spent till now is 4 min(s) 53 sec(s).
60 out of 131 columns are done, total time spent till now is 5 min(s) 53 sec(s).
70 out of 131 columns are done, total time spent till now is 6 min(s) 51 sec(s).
80 out of 131 columns are done, total time spent till now is 7 min(s) 50 sec(s).
90 out of 131 columns are done, total time spent till now is 8 min(s) 49 sec(s).
100 out of 131 columns are done, total time spent till now is 9 min(s) 47 sec(s).
110 out of 131 columns are done, total time spent till now is 10 min(s) 47 sec(s).
120 out of 131 columns are done, total time spent till now is 11 min(s) 45 sec(s).
130 out of 131 columns 

In [28]:
F_summary_daily

,q_2.0_0,q_2.0_1,q_2.0_2,q_2.0_3,q_2.0_4,q_2.0_5,q_2.0_6,q_2.0_7,q_2.0_8,q_2.0_9,...,q_2.0_121,q_2.0_122,q_2.0_123,q_2.0_124,q_2.0_125,q_2.0_126,q_2.0_127,q_2.0_128,q_2.0_129,q_2.0_130
s_3,9.393502,10.111921,13.481367,7.200658,10.696694,6.438678,9.649552,83.876851,15.302399,17.355859,...,7.713781,5.348078,4.062503,2.910221,10.294977,0.999223,1.446548,6.615763,4.260271,15.456232
s_4,10.446906,12.70074,12.338551,8.944858,12.010789,6.977643,10.406803,117.310538,18.028154,20.057092,...,8.018891,5.735078,4.585532,3.425349,11.004562,0.976149,1.543843,6.672004,4.475374,21.773807
s_5,11.527429,13.972869,14.035409,9.20483,13.256235,7.03915,10.989475,104.501537,18.055216,20.800825,...,7.80152,6.497172,5.538305,3.619401,10.855652,1.092249,1.631731,8.698933,4.671835,23.17643
s_6,10.555731,14.032572,13.621803,8.037224,12.766996,7.97344,12.422343,87.272695,18.151929,21.17279,...,10.22779,6.768088,5.22006,3.557942,10.902599,1.081156,1.540283,7.271239,4.941146,16.594153
s_7,10.860096,13.007037,11.88459,8.31072,12.786922,7.336244,12.275378,127.020702,18.186572,21.79813,...,9.354062,7.091177,6.232796,3.750664,15.223728,1.148818,1.635026,5.778301,4.775837,23.096843
s_8,11.157379,12.673885,13.182275,9.828805,12.251023,7.496035,13.01243,121.345387,18.266839,21.786431,...,7.700744,6.428109,6.42061,3.811031,15.687438,0.995373,1.715256,7.014364,5.036519,23.010731
s_9,11.956614,14.424774,13.913389,9.1375,11.822089,8.350954,13.966889,134.35271,19.151353,21.283545,...,10.480957,6.591521,6.666552,4.078838,15.525045,1.193314,1.711562,10.69164,4.745672,15.598099
s_10,12.999045,14.128974,16.778978,9.740335,14.249944,7.146265,12.958002,138.607777,18.499522,21.685684,...,7.665397,6.27776,7.45027,4.039476,11.240646,1.24495,1.755996,11.854923,5.389223,21.862516
s_11,11.162547,15.498607,16.413671,9.710222,15.409769,8.087326,14.235582,118.569981,21.344122,21.376077,...,10.448959,6.872783,7.815917,4.267993,16.253456,1.135717,1.882059,10.913833,5.367017,25.998083
s_12,11.389898,15.233342,18.638792,10.273103,11.970887,7.529057,13.891592,137.575486,20.452687,20.93687,...,9.890993,6.179797,5.711738,4.171921,11.833439,1.213702,2.040288,7.747393,5.260115,26.67698


In [29]:
final_data

,H_Daily,H_Weekly,H_Monthly,Real_VIX,Estimate__VIX
0,0.301912,NaN,NaN,NaN,NaN
1,0.387013,NaN,NaN,NaN,NaN
2,0.310063,NaN,NaN,NaN,NaN
3,0.310326,NaN,NaN,NaN,NaN
4,0.232691,NaN,NaN,NaN,NaN
5,0.186431,NaN,NaN,NaN,NaN
6,0.267487,NaN,NaN,NaN,NaN
7,0.072215,NaN,NaN,NaN,NaN
8,0.308368,NaN,NaN,NaN,NaN
9,0.237425,NaN,NaN,NaN,NaN


In [30]:
# Save temp result
final_data.to_csv('final_data_May2022-Nov2022_daily_08152023.csv')

# Analyze weekly cleaned data

In [31]:
weekly_resized_data

,0,1,2,3,4,5,6,7,8,9,...,121,122,123,124,125,126,127,128,129,130
0,5.752994,5.970206,8.557028,2.779768,5.639659,13.171149,5.580517,0.23631,1.411071,0.284226,...,14.628843,1.491534,3.46287,3.673909,0.414842,12.494416,6.5393,5.01426,0.411395,0.0
1,33.413222,0.001982,0.000212,22.53126,3.531361,85.100356,0.267426,13.341419,8.55685,8.377972,...,86.966666,21.215247,25.067593,64.783508,1.439162,0.893057,0.40556,0.905215,1.588139,22.842914
2,25.503274,1.421449,3.679338,19.828536,18.762537,8.307518,12.85593,30.841752,9.38259,5.81495,...,0.000314,0.788562,0.413831,31.59356,4.214658,0.000058,33.323274,39.306031,1.51289,1.740986
3,77.731886,33.715762,13.329835,0.116769,107.009487,5.894893,2.881878,0.480029,9.38259,3.775956,...,24.698654,3.577201,17.703739,0.432987,0.398474,0.38314,8.788849,19.066371,6.386508,30.527634
4,1.57667,0.026624,23.559571,0.214914,1.539153,34.980546,4.052083,1.512848,52.635276,2.896063,...,57.920365,0.000056,0.027532,6.063023,15.284759,14.409722,47.022371,1.2065,0.364233,3.323953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6605,5.413135,23.348845,5.875496,1.549912,23.697807,2.83922,1.812147,6.547971,1.852901,1.857592,...,3.608703,1.544732,0.025442,9.552949,152.278487,3.343653,0.016282,0.404322,3.498663,0.0
6606,31.49,1.464195,2.72496,1.409163,1.511596,3.917008,0.017403,18.797638,71.206041,5.095757,...,28.56779,18.774283,0.261546,5.97008,18.895094,0.392914,3.520462,0.000101,0.388643,0.34842
6607,87.880289,3.647274,28.362581,5.847293,5.939227,10.24276,73.298547,72.68405,3.547382,29.743824,...,12.389503,41.379097,1.596172,5.478482,0.469756,0.374545,0.57833,0.864776,0.001433,0.000864
6608,116.606859,2.907119,43.076684,0.515946,117.400615,0.221835,2.879914,17.577383,34.531739,12.328013,...,0.434701,0.642632,13.980805,0.010559,0.373137,0.402336,0.372576,0.00091,4.025076,0.396215


In [32]:
# Define parameters
n_sample = modified_weekly_data
scales = np.unique(np.logspace(np.log10(3), np.log10((n_sample+1)/10), 50).astype(int))
# qs = list(np.linspace(-10, 10, num=41, endpoint=True))
qs = [2.0]

weekly_cumsum = weekly_resized_data.cumsum(axis=0)
weekly_cumsum

,0,1,2,3,4,5,6,7,8,9,...,121,122,123,124,125,126,127,128,129,130
0,5.752994,5.970206,8.557028,2.779768,5.639659,13.171149,5.580517,0.23631,1.411071,0.284226,...,14.628843,1.491534,3.46287,3.673909,0.414842,12.494416,6.5393,5.01426,0.411395,0.0
1,39.166216,5.972188,8.557239,25.311029,9.17102,98.271505,5.847943,13.577729,9.967921,8.662198,...,101.59551,22.706782,28.530463,68.457417,1.854004,13.387474,6.94486,5.919475,1.999534,22.842914
2,64.66949,7.393637,12.236577,45.139564,27.933556,106.579022,18.703873,44.419481,19.350511,14.477147,...,101.595823,23.495343,28.944294,100.050977,6.068662,13.387532,40.268134,45.225507,3.512424,24.5839
3,142.401377,41.1094,25.566413,45.256333,134.943043,112.473915,21.585751,44.89951,28.733101,18.253103,...,126.294477,27.072544,46.648033,100.483964,6.467136,13.770672,49.056983,64.291878,9.898931,55.111534
4,143.978046,41.136024,49.125984,45.471248,136.482196,147.454462,25.637834,46.412358,81.368377,21.149166,...,184.214842,27.072601,46.675564,106.546987,21.751895,28.180395,96.079354,65.498378,10.263164,58.435488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6605,81433.783788,78806.634312,85863.22552,94050.883859,88895.02291,78365.479005,76625.582487,97281.451871,120587.446519,143912.537623,...,82548.783333,76684.529431,73776.835345,52096.717361,46300.45168,35724.115017,26968.425251,25166.475554,26915.770943,34884.018566
6606,81465.273788,78808.098507,85865.95048,94052.293022,88896.534505,78369.396013,76625.59989,97300.249509,120658.65256,143917.63338,...,82577.351123,76703.303714,73777.096891,52102.687442,46319.346774,35724.507931,26971.945714,25166.475655,26916.159586,34884.366986
6607,81553.154077,78811.745781,85894.313061,94058.140315,88902.473733,78379.638774,76698.898438,97372.933559,120662.199941,143947.377204,...,82589.740626,76744.68281,73778.693064,52108.165924,46319.81653,35724.882475,26972.524044,25167.340431,26916.161019,34884.36785
6608,81669.760937,78814.6529,85937.389746,94058.656261,89019.874348,78379.860609,76701.778352,97390.510942,120696.73168,143959.705217,...,82590.175327,76745.325442,73792.673869,52108.176483,46320.189668,35725.284812,26972.89662,25167.34134,26920.186096,34884.764065


In [33]:
# Create DMA flucuation function dataframe
F_summary_columns = []
for q in qs:
    for col in range(num_obs):
        F_summary_columns.append('q_'+str(q)+'_'+str(col))

F_summary_index = []
for scale in scales:
    F_summary_index.append('s_'+str(scale))

F_summary_weekly = pd.DataFrame(index=F_summary_index, columns=F_summary_columns)
F_summary_weekly

,q_2.0_0,q_2.0_1,q_2.0_2,q_2.0_3,q_2.0_4,q_2.0_5,q_2.0_6,q_2.0_7,q_2.0_8,q_2.0_9,...,q_2.0_121,q_2.0_122,q_2.0_123,q_2.0_124,q_2.0_125,q_2.0_126,q_2.0_127,q_2.0_128,q_2.0_129,q_2.0_130
s_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
starttime_total = time.time()

log10_scale = np.log10(scales)

for col in range(num_obs):
    ana_temp = weekly_cumsum[col]
    
    for scale in scales:
        # Calculate moving average
        mean_temp = []
        for i in range(n_sample-scale+1):
            mean_temp.append(np.mean(ana_temp[i:i+scale]))
        mean_temp.reverse()
        
        # Detrend
        diff_temp_subtractor = ana_temp[:scale-2:-1].values
        diff_temp = np.subtract(np.array(mean_temp), diff_temp_subtractor)
        
        # Slice into non-overlapping segments
        segs_temp = pd.DataFrame(np.resize(diff_temp, (len(diff_temp) // scale, scale))).T
        
        # Calculate fluctuation variance
        F_seg_sq_temp = np.array(np.var(segs_temp).mean(axis=0))
        
        # Calculate the qth order fluctuation function
        for q in qs:
            if q != 0.0:
                F_temp =  ((1 / scale) * (F_seg_sq_temp ** (q / 2)).sum()) ** (1 / q)
            else:
                F_temp = np.exp(np.log(F_seg_sq_temp ** (1 / 2)).mean()) # L’Hôspital’s rule when q = 0
            
            F_summary_weekly.loc[str('s_'+str(scale))][str('q_'+str(q))+'_'+str(col)] = F_temp
       
    # Calculate Hurst exponent
    log10_F = np.log10(np.array(F_summary_weekly[str('q_'+str(q))+'_'+str(col)]).astype(float))
    hurst_hat = np.polyfit(log10_scale, log10_F, 1)[0]
    final_data['H_Weekly'][col] = hurst_hat
    
    if (col+1) % 10 == 0:
        endtime_total = time.time()
        total_time = endtime_total - starttime_total
        print(f'{col+1} out of {num_obs} columns are done,', f'total time spent till now is',
              f'{int((endtime_total-starttime_total)//60)} min(s) {int((endtime_total-starttime_total) % 60)} sec(s).')

endtime_total = time.time()
total_time = endtime_total - starttime_total
print(f'All done! Total time spent is',f'{int((endtime_total-starttime_total)//60)}',
      f'min(s) {int((endtime_total-starttime_total) % 60)} sec(s).')

10 out of 131 columns are done, total time spent till now is 5 min(s) 55 sec(s).
20 out of 131 columns are done, total time spent till now is 11 min(s) 48 sec(s).
30 out of 131 columns are done, total time spent till now is 17 min(s) 41 sec(s).
40 out of 131 columns are done, total time spent till now is 23 min(s) 34 sec(s).
50 out of 131 columns are done, total time spent till now is 29 min(s) 25 sec(s).
60 out of 131 columns are done, total time spent till now is 35 min(s) 20 sec(s).
70 out of 131 columns are done, total time spent till now is 41 min(s) 11 sec(s).
80 out of 131 columns are done, total time spent till now is 47 min(s) 7 sec(s).
90 out of 131 columns are done, total time spent till now is 52 min(s) 58 sec(s).
100 out of 131 columns are done, total time spent till now is 58 min(s) 54 sec(s).
110 out of 131 columns are done, total time spent till now is 64 min(s) 45 sec(s).
120 out of 131 columns are done, total time spent till now is 70 min(s) 37 sec(s).
130 out of 131 

In [35]:
F_summary_weekly

,q_2.0_0,q_2.0_1,q_2.0_2,q_2.0_3,q_2.0_4,q_2.0_5,q_2.0_6,q_2.0_7,q_2.0_8,q_2.0_9,...,q_2.0_121,q_2.0_122,q_2.0_123,q_2.0_124,q_2.0_125,q_2.0_126,q_2.0_127,q_2.0_128,q_2.0_129,q_2.0_130
s_3,11.138915,7.880901,10.244182,10.036645,10.074438,10.339632,9.583514,38.325637,48.424868,36.55127,...,14.346125,16.48739,13.123382,5.658398,7.014752,5.225899,3.972075,5.704724,5.544647,7.493329
s_4,10.360264,9.033101,10.417573,11.668921,11.263705,11.606616,10.186851,53.136034,39.776772,53.975949,...,14.850094,17.843772,14.375949,6.562213,7.168026,5.560003,5.602141,5.312933,6.605396,10.423826
s_5,10.627151,9.758735,11.191785,11.170597,12.617136,11.093892,11.32924,47.578232,38.309262,55.561742,...,18.91627,17.208978,13.293168,7.479796,7.49788,5.887998,6.51697,5.846109,6.837276,11.405517
s_6,11.386914,9.797148,11.282354,12.270843,12.159274,11.609913,11.749379,40.04289,55.410246,54.619433,...,15.966464,18.903068,18.488497,7.559383,7.57147,6.117767,7.202028,6.168042,6.446171,8.850358
s_7,12.22142,9.49907,10.571507,11.612005,12.343422,12.413483,12.455917,57.67571,59.898899,56.001038,...,17.383696,18.740223,19.978999,7.360041,8.691949,7.531701,5.544464,5.133219,6.36117,11.268918
s_8,11.862059,9.992838,11.875144,12.635281,12.644018,11.929194,11.681612,55.005551,60.487496,52.020281,...,15.718715,19.26099,20.962804,7.36165,8.900347,7.93436,6.066762,5.707606,8.409499,11.672186
s_9,13.207513,10.648345,11.953988,11.940348,12.498073,12.363646,12.625591,60.870973,55.186421,40.630551,...,21.194454,19.086833,15.029731,7.71314,9.192141,8.542159,7.918116,8.167469,6.572246,7.820677
s_10,14.305173,10.70106,12.538727,13.354895,13.679667,11.751706,11.648616,62.667914,58.710952,49.614746,...,19.34132,21.646519,20.690817,8.275758,7.997145,6.452749,7.781369,9.186734,8.642708,10.912267
s_11,14.145051,11.103487,13.247346,14.152146,14.137079,12.762974,11.66883,53.6335,63.325498,61.325796,...,21.338582,21.950026,20.486611,8.218818,9.266255,8.458967,7.505759,7.384775,6.568394,12.206675
s_12,15.196876,10.968699,13.837836,13.924742,12.851358,11.96015,13.214585,62.158982,56.196275,48.783277,...,16.382692,18.719067,21.603316,8.39752,8.926152,7.005648,7.943271,8.469272,8.695847,13.218878


In [36]:
final_data

,H_Daily,H_Weekly,H_Monthly,Real_VIX,Estimate__VIX
0,0.301912,0.297286,NaN,NaN,NaN
1,0.387013,0.435357,NaN,NaN,NaN
2,0.310063,0.427269,NaN,NaN,NaN
3,0.310326,0.439632,NaN,NaN,NaN
4,0.232691,0.402926,NaN,NaN,NaN
5,0.186431,0.409892,NaN,NaN,NaN
6,0.267487,0.382694,NaN,NaN,NaN
7,0.072215,0.141865,NaN,NaN,NaN
8,0.308368,0.173405,NaN,NaN,NaN
9,0.237425,0.197462,NaN,NaN,NaN


In [37]:
# Save temp result
final_data.to_csv('final_data_May2022-Nov2022_weekly_08152023.csv')

# Analyze monthly cleaned data

In [38]:
monthly_resized_data

,0,1,2,3,4,5,6,7,8,9,...,121,122,123,124,125,126,127,128,129,130
0,71.111366,0.209118,128.50794,16.787078,72.324201,187.233906,38.30059,246.579329,45.34947,1.644969,...,0.129363,23.028574,13.78011,3.808376,10.773752,0.005754,0.019696,3.986904,0.0,0.38653
1,8.007312,5.630619,32.931311,9.146669,62.709439,5.580177,8.903187,0.056713,38.221926,62.739891,...,0.498446,16.294054,1.717811,18.124947,18.243348,1.360851,1.621956,0.293114,0.000059,0.446426
2,8.640116,70.671399,8.384588,529.45694,117.497259,23.89302,0.969221,23.378483,304.607129,26.337527,...,18.480609,11.212626,3.984852,1.710135,8.34365,7.543762,1.689269,0.018143,3.434759,0.001497
3,0.255835,5.711316,3.708418,189.947924,46.122058,35.622106,59.045768,160.674148,26.695085,9.415659,...,0.446785,0.0,0.338013,0.000063,3.546191,2.665439,0.0,27.971905,1.608056,1.565532
4,27.702594,27.830699,463.910376,45.345558,37.082814,45.541844,38.027716,36.450201,24.84114,3.809879,...,3.925007,11.701958,4.970166,8.700266,9.657414,10.814294,0.026823,0.001571,3.696914,0.000959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29079,5.413135,23.348845,5.875496,1.549912,23.697807,2.83922,1.812147,6.547971,1.852901,1.857592,...,3.608703,1.544732,0.025442,9.552949,152.278487,3.343653,0.016282,0.404322,3.498663,0.0
29080,31.49,1.464195,2.72496,1.409163,1.511596,3.917008,0.017403,18.797638,71.206041,5.095757,...,28.56779,18.774283,0.261546,5.97008,18.895094,0.392914,3.520462,0.000101,0.388643,0.34842
29081,87.880289,3.647274,28.362581,5.847293,5.939227,10.24276,73.298547,72.68405,3.547382,29.743824,...,12.389503,41.379097,1.596172,5.478482,0.469756,0.374545,0.57833,0.864776,0.001433,0.000864
29082,116.606859,2.907119,43.076684,0.515946,117.400615,0.221835,2.879914,17.577383,34.531739,12.328013,...,0.434701,0.642632,13.980805,0.010559,0.373137,0.402336,0.372576,0.00091,4.025076,0.396215


In [39]:
# Define parameters
n_sample = modified_monthly_data
scales = np.unique(np.logspace(np.log10(3), np.log10((n_sample+1)/10), 50).astype(int))
# qs = list(np.linspace(-10, 10, num=41, endpoint=True))
qs = [2.0]

monthly_cumsum = monthly_resized_data.cumsum(axis=0)
monthly_cumsum

,0,1,2,3,4,5,6,7,8,9,...,121,122,123,124,125,126,127,128,129,130
0,71.111366,0.209118,128.50794,16.787078,72.324201,187.233906,38.30059,246.579329,45.34947,1.644969,...,0.129363,23.028574,13.78011,3.808376,10.773752,0.005754,0.019696,3.986904,0.0,0.38653
1,79.118678,5.839737,161.439251,25.933746,135.03364,192.814083,47.203777,246.636042,83.571395,64.384859,...,0.627809,39.322628,15.497922,21.933323,29.0171,1.366605,1.641653,4.280018,0.000059,0.832956
2,87.758794,76.511137,169.823839,555.390687,252.530899,216.707103,48.172998,270.014525,388.178525,90.722387,...,19.108418,50.535254,19.482773,23.643458,37.36075,8.910366,3.330922,4.298161,3.434818,0.834453
3,88.014629,82.222452,173.532257,745.338611,298.652957,252.32921,107.218765,430.688673,414.87361,100.138046,...,19.555203,50.535254,19.820786,23.643521,40.906941,11.575805,3.330922,32.270066,5.042874,2.399986
4,115.717223,110.053152,637.442632,790.684169,335.735771,297.871053,145.246481,467.138874,439.71475,103.947925,...,23.48021,62.237212,24.790952,32.343787,50.564355,22.3901,3.357745,32.271637,8.739788,2.400944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29079,724394.198463,714412.560269,714226.941092,681456.492909,631652.275488,600418.186262,569648.561363,544256.282852,516622.133594,521164.786714,...,548020.182836,533276.048474,512543.919578,481182.022368,451128.306211,439101.451959,422170.098004,401268.991439,387249.173593,390378.944719
29080,724425.688463,714414.024463,714229.666052,681457.902072,631653.787084,600422.10327,569648.578766,544275.08049,516693.339634,521169.882471,...,548048.750626,533294.822757,512544.181125,481187.992448,451147.201305,439101.844873,422173.618467,401268.99154,387249.562236,390379.293139
29081,724513.568753,714417.671737,714258.028633,681463.749365,631659.726311,600432.346031,569721.877313,544347.764539,516696.887016,521199.626295,...,548061.140129,533336.201853,512545.777297,481193.47093,451147.671062,439102.219418,422174.196797,401269.856316,387249.56367,390379.294003
29082,724630.175612,714420.578856,714301.105317,681464.265311,631777.126926,600432.567866,569724.757228,544365.341922,516731.418755,521211.954308,...,548061.57483,533336.844485,512559.758102,481193.481489,451148.044199,439102.621754,422174.569373,401269.857225,387253.588746,390379.690218


In [42]:
# Create DMA flucuation function dataframe
F_summary_columns = []
for q in qs:
    for col in range(num_obs):
        F_summary_columns.append('q_'+str(q)+'_'+str(col))

F_summary_index = []
for scale in scales:
    F_summary_index.append('s_'+str(scale))

F_summary_monthly = pd.DataFrame(index=F_summary_index, columns=F_summary_columns)
F_summary_monthly

,q_2.0_0,q_2.0_1,q_2.0_2,q_2.0_3,q_2.0_4,q_2.0_5,q_2.0_6,q_2.0_7,q_2.0_8,q_2.0_9,...,q_2.0_121,q_2.0_122,q_2.0_123,q_2.0_124,q_2.0_125,q_2.0_126,q_2.0_127,q_2.0_128,q_2.0_129,q_2.0_130
s_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s_16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
starttime_total = time.time()

log10_scale = np.log10(scales)

for col in range(num_obs):
    ana_temp = monthly_cumsum[col]
    
    for scale in scales:
        # Calculate moving average
        mean_temp = []
        for i in range(n_sample-scale+1):
            mean_temp.append(np.mean(ana_temp[i:i+scale]))
        mean_temp.reverse()
        
        # Detrend
        diff_temp_subtractor = ana_temp[:scale-2:-1].values
        diff_temp = np.subtract(np.array(mean_temp), diff_temp_subtractor)
        
        # Slice into non-overlapping segments
        segs_temp = pd.DataFrame(np.resize(diff_temp, (len(diff_temp) // scale, scale))).T
        
        # Calculate fluctuation variance
        F_seg_sq_temp = np.array(np.var(segs_temp).mean(axis=0))
        
        # Calculate the qth order fluctuation function
        for q in qs:
            if q != 0.0:
                F_temp =  ((1 / scale) * (F_seg_sq_temp ** (q / 2)).sum()) ** (1 / q)
            else:
                F_temp = np.exp(np.log(F_seg_sq_temp ** (1 / 2)).mean()) # L’Hôspital’s rule when q = 0
            
            F_summary_monthly.loc[str('s_'+str(scale))][str('q_'+str(q))+'_'+str(col)] = F_temp
       
    # Calculate Hurst exponent
    log10_F = np.log10(np.array(F_summary_monthly[str('q_'+str(q))+'_'+str(col)]).astype(float))
    hurst_hat = np.polyfit(log10_scale, log10_F, 1)[0]
    final_data['H_Monthly'][col] = hurst_hat
    
    if (col+1) % 5 == 0:
        endtime_total = time.time()
        total_time = endtime_total - starttime_total
        print(f'{col+1} out of {num_obs} columns are done,', f'total time spent till now is',
              f'{int((endtime_total-starttime_total)//60)} min(s) {int((endtime_total-starttime_total) % 60)} sec(s).')

endtime_total = time.time()
total_time = endtime_total - starttime_total
print(f'All done! Total time spent is',f'{int((endtime_total-starttime_total)//60)}',
      f'min(s) {int((endtime_total-starttime_total) % 60)} sec(s).')

5 out of 131 columns are done, total time spent till now is 16 min(s) 10 sec(s).
10 out of 131 columns are done, total time spent till now is 32 min(s) 32 sec(s).
15 out of 131 columns are done, total time spent till now is 48 min(s) 49 sec(s).
20 out of 131 columns are done, total time spent till now is 65 min(s) 49 sec(s).
25 out of 131 columns are done, total time spent till now is 82 min(s) 6 sec(s).
30 out of 131 columns are done, total time spent till now is 98 min(s) 22 sec(s).
35 out of 131 columns are done, total time spent till now is 117 min(s) 38 sec(s).
40 out of 131 columns are done, total time spent till now is 133 min(s) 50 sec(s).
45 out of 131 columns are done, total time spent till now is 150 min(s) 2 sec(s).
50 out of 131 columns are done, total time spent till now is 172 min(s) 4 sec(s).
55 out of 131 columns are done, total time spent till now is 188 min(s) 17 sec(s).
60 out of 131 columns are done, total time spent till now is 204 min(s) 36 sec(s).
65 out of 131 

In [44]:
F_summary_monthly

,q_2.0_0,q_2.0_1,q_2.0_2,q_2.0_3,q_2.0_4,q_2.0_5,q_2.0_6,q_2.0_7,q_2.0_8,q_2.0_9,...,q_2.0_121,q_2.0_122,q_2.0_123,q_2.0_124,q_2.0_125,q_2.0_126,q_2.0_127,q_2.0_128,q_2.0_129,q_2.0_130
s_3,30.718358,27.19905,29.3161,29.889754,22.978522,22.800318,26.124851,22.691892,26.050689,20.811332,...,95.381607,75.731697,78.572857,95.197333,75.195461,78.103323,94.714083,74.87913,77.419385,94.395147
s_4,32.898955,29.574651,32.765057,28.66774,27.177711,24.662399,26.324859,29.525376,22.899026,28.471437,...,95.621439,78.445467,95.516415,78.219497,94.967178,77.844579,94.814481,77.39719,94.25656,77.232973
s_5,33.922038,33.84962,30.459358,33.348484,25.636311,29.577396,26.974069,27.586269,22.851855,29.485758,...,80.388005,109.337174,90.784145,82.923181,104.617524,79.684058,108.725028,89.659188,81.681559,103.928359
s_6,32.808322,30.842284,37.127083,32.0979,26.230103,30.957752,26.649272,25.102791,29.934242,29.183672,...,88.946446,83.667678,110.420278,88.634697,82.972913,110.030569,88.085718,82.203917,109.313036,87.629086
s_8,36.74412,35.076905,34.150249,32.791769,32.242589,30.329208,26.508204,31.274733,32.287867,28.325516,...,87.696128,80.031834,117.374426,107.784205,86.871555,79.190314,116.780395,107.094863,86.051135,78.280069
s_9,36.407369,37.133228,37.109721,34.715124,25.717767,25.831663,26.447608,33.579315,30.316763,24.150811,...,86.003497,94.219981,103.635243,111.818849,116.954184,116.980166,108.489531,83.780602,82.487107,84.009512
s_10,35.409756,35.017063,33.849055,36.94613,32.625281,30.078514,27.260613,34.62839,31.763987,28.025671,...,114.000798,104.370423,90.16138,83.182811,114.512688,113.336361,103.56844,88.79085,81.597996,113.608096
s_12,33.34525,32.70382,38.463215,39.35637,32.343467,30.04771,27.037932,34.90709,31.480223,27.457248,...,89.051535,83.489006,113.905148,117.517453,113.326009,101.161024,87.988924,81.810381,112.487105,116.630977
s_14,32.782037,41.356283,33.112998,39.982485,29.460867,33.066835,31.580098,35.107959,31.088752,29.200874,...,86.081164,123.362709,95.951796,115.374054,107.637699,81.770165,118.009991,84.464402,121.883009,94.411615
s_16,40.620516,35.681068,40.982079,34.489639,32.952922,34.933795,26.986942,31.153324,31.044918,35.335007,...,84.225597,116.46902,113.808967,92.426862,123.438629,80.324992,103.85181,123.451941,82.391829,114.660669


In [45]:
final_data

,H_Daily,H_Weekly,H_Monthly,Real_VIX,Estimate__VIX
0,0.301912,0.297286,0.309182,NaN,NaN
1,0.387013,0.435357,0.320339,NaN,NaN
2,0.310063,0.427269,0.301494,NaN,NaN
3,0.310326,0.439632,0.29827,NaN,NaN
4,0.232691,0.402926,0.318781,NaN,NaN
5,0.186431,0.409892,0.308863,NaN,NaN
6,0.267487,0.382694,0.301159,NaN,NaN
7,0.072215,0.141865,0.266718,NaN,NaN
8,0.308368,0.173405,0.260906,NaN,NaN
9,0.237425,0.197462,0.274218,NaN,NaN


In [46]:
# Save temp result
final_data.to_csv('final_data_May2022-Nov2022_monthly_08152023.csv')

# Add VIX into the DataFrame

In [47]:
final_data = pd.read_csv('final_data_May2022-Nov2022_monthly_08152023.csv', index_col=0)

In [48]:
# Retrieve daily VIX data from Github repository
datasource_vix = 'https://raw.githubusercontent.com/y3kiys4/wqu_capstone_june2023_grp3626/main/VIX_1d_May2022-Nov2022.csv'

vix = pd.read_csv(datasource_vix)
print(vix)

           DATE  VIXCLS
0    2022-05-02   32.34
1    2022-05-03   29.25
2    2022-05-04   25.42
3    2022-05-05   31.20
4    2022-05-06   30.19
5    2022-05-09   34.75
6    2022-05-10   32.99
7    2022-05-11   32.56
8    2022-05-12   31.77
9    2022-05-13   28.87
10   2022-05-16   27.47
11   2022-05-17   26.10
12   2022-05-18   30.96
13   2022-05-19   29.35
14   2022-05-20   29.43
15   2022-05-23   28.48
16   2022-05-24   29.45
17   2022-05-25   28.37
18   2022-05-26   27.50
19   2022-05-27   25.72
20   2022-05-30   26.54
21   2022-05-31   26.19
22   2022-06-01   25.69
23   2022-06-02   24.72
24   2022-06-03   24.79
25   2022-06-06   25.07
26   2022-06-07   24.02
27   2022-06-08   23.96
28   2022-06-09   26.09
29   2022-06-10   27.75
30   2022-06-13   34.02
31   2022-06-14   32.69
32   2022-06-15   29.62
33   2022-06-16   32.95
34   2022-06-17   31.13
35   2022-06-20   31.03
36   2022-06-21   30.19
37   2022-06-22   28.95
38   2022-06-23   29.05
39   2022-06-24   27.23
40   2022-06-27 

In [49]:
# Align the number of VIX observations with that of Hurst results
vix1=vix.drop(vix.index[:(len(vix)-num_obs)])
vix1

,DATE,VIXCLS
22,2022-06-01,25.69
23,2022-06-02,24.72
24,2022-06-03,24.79
25,2022-06-06,25.07
26,2022-06-07,24.02
27,2022-06-08,23.96
28,2022-06-09,26.09
29,2022-06-10,27.75
30,2022-06-13,34.02
31,2022-06-14,32.69


In [50]:
# Reset the index
vix2 = pd.DataFrame(vix1['VIXCLS'].values.tolist())
vix2

,0
0,25.69
1,24.72
2,24.79
3,25.07
4,24.02
5,23.96
6,26.09
7,27.75
8,34.02
9,32.69


In [53]:
final_data['Real_VIX'] = vix2
final_data

# Save temp result
final_data.to_csv('final_data_May2022-Nov2022_realvix_08152023.csv')

# Use machine learning to find the relatioship between H and VIX

In [55]:
final_data = pd.read_csv('final_data_May2022-Nov2022_realvix_08152023.csv', index_col=0)
final_data

,H_Daily,H_Weekly,H_Monthly,Real_VIX,Estimate__VIX
0,0.301912,0.297286,0.309182,25.69,NaN
1,0.387013,0.435357,0.320339,24.72,NaN
2,0.310063,0.427269,0.301494,24.79,NaN
3,0.310326,0.439632,0.298270,25.07,NaN
4,0.232691,0.402926,0.318781,24.02,NaN
5,0.186431,0.409892,0.308863,23.96,NaN
6,0.267487,0.382694,0.301159,26.09,NaN
7,0.072215,0.141865,0.266718,27.75,NaN
8,0.308368,0.173405,0.260906,34.02,NaN
9,0.237425,0.197462,0.274218,32.69,NaN


# Future Study

In [ ]:
# Weekly data availability fluctuates
available_weekly.plot()

In [ ]:
# Daily data ability fluctuates
available_daily.groupby(['day_of_week']).plot()

In [ ]:
available_monthly.plot()

#### Investigate negative H_Daily values

In [ ]:
daily_cumsum[126].plot()

In [ ]:
# Define parameters
n_sample = avg_daily
scales = np.unique(np.logspace(np.log10(3), np.log10((n_sample+1)/10), 50).astype(int))
num_obs = 131

In [ ]:
# Prepare the final dataframe to analyze
final_data_cols = ['H_Daily', 'H_Weekly', 'H_Monthly', 'Real_VIX', 'Estimate__VIX']
final_data1 = pd.DataFrame(index=np.arange(num_obs), columns=final_data_cols)
final_data1

In [ ]:
# Create DMA flucuation function dataframe
F_summary_columns = []
for q in qs:
    for col in [11, 126]:
        F_summary_columns.append('q_'+str(q)+'_'+str(col))

F_summary_index = []
for scale in scales:
    F_summary_index.append('s_'+str(scale))

F_summary_daily1 = pd.DataFrame(index=F_summary_index, columns=F_summary_columns)
F_summary_daily1

In [ ]:
starttime_total = time.time()

log10_scale = np.log10(scales)

for col in [11, 126]:
    ana_temp = daily_cumsum[col]
    
    for scale in scales:
        # Calculate moving average
        mean_temp = []
        for i in range(n_sample-scale+1):
            mean_temp.append(np.mean(ana_temp[i:i+scale]))
        mean_temp.reverse()
        
        # Detrend
        diff_temp_subtractor = ana_temp[:scale-2:-1].values
        diff_temp = np.subtract(np.array(mean_temp), diff_temp_subtractor)
        
        # Slice into non-overlapping segments
        segs_temp = pd.DataFrame(np.resize(diff_temp, (len(diff_temp) // scale, scale))).T
        
        # Calculate fluctuation variance
        F_seg_sq_temp = np.array(np.var(segs_temp).mean(axis=0))
        
        # Calculate the qth order fluctuation function
        for q in qs:
            if q != 0.0:
                F_temp =  ((1 / scale) * (F_seg_sq_temp ** (q / 2)).sum()) ** (1 / q)
            else:
                F_temp = np.exp(np.log(F_seg_sq_temp ** (1 / 2)).mean()) # L’Hôspital’s rule when q = 0
            
            F_summary_daily1.loc[str('s_'+str(scale))][str('q_'+str(q))+'_'+str(col)] = F_temp
       
    # Calculate Hurst exponent
    log10_F = np.log10(np.array(F_summary_daily1[str('q_'+str(q))+'_'+str(col)]).astype(float))
    hurst_hat = np.polyfit(log10_scale, log10_F, 1)[0]
    final_data1['H_Daily'][col] = hurst_hat
    
    if (col+1) % 10 == 0:
        endtime_total = time.time()
        total_time = endtime_total - starttime_total
        print(f'{col+1} out of {num_obs} columns are done,', f'total time spent till now is',
              f'{int((endtime_total-starttime_total)//60)} min(s) {int((endtime_total-starttime_total) % 60)} sec(s).')

endtime_total = time.time()
total_time = endtime_total - starttime_total
print(f'All done! Total time spent is',f'{int((endtime_total-starttime_total)//60)}',
      f'min(s) {int((endtime_total-starttime_total) % 60)} sec(s).')

In [ ]:
F_summary_daily1

In [ ]:
final_data1

In [ ]:
# Log-log plot and calculate H
log10_lag = np.log10(scales)
log10_F = np.log10(np.array(F_summary_daily1).astype(float)) 
hurst_hat = np.polyfit(log10_lag, log10_F, 1)[0]
# hurst_hat = np.polyfit(log10_lag[:29], log10_F[:29], 1)[0]

plt.plot(log10_lag, log10_F, marker="o", markersize=3, markeredgecolor="red", markerfacecolor="green")
# plt.plot(log10_lag[:29], log10_F[:29], marker="o", markersize=3, markeredgecolor="red", markerfacecolor="green")
plt.xlabel('Log_s')
plt.ylabel('Log_F(s)')
plt.show()

How to fix? Method one: delete the negative H_Daily values

In [ ]:
# # Remove rows containing negative daily Hurst exponents (index 11 and 126)
# indexAge = final_data[final_data['H_Daily'] < 0].index
# final_data.drop(indexAge, inplace=True)
# final_data

# # Reset index
# final_for_ml = pd.DataFrame(final_data.to_numpy(), columns=final_data_cols)

How to fix? Method one: only use uptrended data

In [ ]:
# Log-log plot and calculate H
log10_lag = np.log10(scales)
log10_F = np.log10(np.array(F_summary_daily1).astype(float)) 
hurst_hat = np.polyfit(log10_lag[:29], log10_F[:29], 1)[0]

plt.plot(log10_lag[:29], log10_F[:29], marker="o", markersize=3, markeredgecolor="red", markerfacecolor="green")
plt.xlabel('Log_s')
plt.ylabel('Log_F(s)')
plt.show()